<div style="text-align: center">
  <img alt="AIcrowd" src="https://gitlab.aicrowd.com/jyotish/pricing-game-notebook-scripts/raw/master/pricing-game-banner.png">
</div>

# How to use this notebook 📝

1. **Copy the notebook**. This is a shared template and any edits you make here will not be saved. _You should copy it into your own drive folder._ For this, click the "File" menu (top-left), then "Save a Copy in Drive". You can edit your copy however you like.
2. **Link it to your AICrowd account**. In order to submit your code to AICrowd, you need to provide your account's API key (see [_"Configure static variables"_](#static-var) for details).
3. **Stick to the function definitions**. The submission to AICrowd will look for the pre-defined function names:
  - `fit_model`
  - `save_model`
  - `load_model`
  - `predict_expected_claim`
  - `predict_premium`

    Anything else you write outside of these functions will not be part of the final submission (including constants and utility functions), so make sure everything is defined within them, except for:
4. **Define your preprocessing**. In addition to the functions above, anything in the cell labelled [_"Define your data preprocessing"_](#data-preprocessing) will also be imported into your final submission. 

# Your pricing model 🕵️

In this notebook, you can play with the data, and define and train your pricing model. You can then directly submit it to the AICrowd, with some magic code at the end.

### Baseline logistic regression 💪
You can also play with a baseline logistic regression model [implemented here](https://colab.research.google.com/drive/1iDgDgWUw9QzOkbTYjeyY3i3DGuCoghs3?usp=sharing). 

# Setup the notebook 🛠

In [ ]:
!bash <(curl -sL https://gitlab.aicrowd.com/jyotish/pricing-game-notebook-scripts/raw/master/python/setup.sh)
from aicrowd_helpers import *

⚙️ Installing AIcrowd utilities...
  Running command git clone -q https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli /tmp/pip-req-build-z3voe7_5
✅ Installed AIcrowd utilities


# Configure static variables 📎
<a name="static-var"></a>

In order to submit using this notebook, you must visit this URL https://aicrowd.com/participants/me and copy your API key. 

Then you must set the value of `AICROWD_API_KEY` wuth the value.

In [ ]:
import sklearn

class Config:
  TRAINING_DATA_PATH = 'training.csv'
  MODEL_OUTPUT_PATH = 'model.pkl'
  AICROWD_API_KEY = 'eaab81e0ad4d64a6b0e7ec99a89205f6'  # You can get the key from https://aicrowd.com/participants/me
  ADDITIONAL_PACKAGES = [
    'numpy',  # you can define versions as well, numpy==0.19.2
    'pandas',
    'scikit-learn==' + sklearn.__version__,
    "tqdm",
    "xgboost"
    ""
  ]

# Download dataset files 💾

In [ ]:
# Make sure to offically join the challenge and accept the challenge rules! Otherwise you will not be able to download the data
%download_aicrowd_dataset

💾 Downloading dataset...
Verifying API Key...
API Key valid
Saved API Key successfully!
✅ Downloaded dataset


# Packages 🗃

<a name="packages"></a>

Import here all the packages you need to define your model. **You will need to include all of these packages in `Config.ADDITIONAL_PACKAGES` for your code to run properly once submitted.**

In [ ]:
%%track_imports

import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import xgboost as xgb
import itertools

In [ ]:
import importlib
import global_imports
importlib.reload(global_imports)
from global_imports import *  # do not change this

# Loading the data 📲

In [ ]:
df = pd.read_csv(Config.TRAINING_DATA_PATH)
X_train = df.drop(columns=['claim_amount'])
y_train = df['claim_amount']

## How does the data look like? 🔍

In [ ]:
X_train.sample(n=4)

,id_policy,year,pol_no_claims_discount,pol_coverage,pol_duration,pol_sit_duration,pol_pay_freq,pol_payd,pol_usage,drv_sex1,drv_age1,drv_age_lic1,drv_drv2,drv_sex2,drv_age2,drv_age_lic2,vh_make_model,vh_age,vh_fuel,vh_type,vh_speed,vh_value,vh_weight,population,town_surface_area
42222,PL072706,1.0,0.166,Max,7,5,Yearly,No,WorkPrivate,M,29.0,11.0,No,0,NaN,NaN,kpciudedjlrqsfte,5.0,Gasoline,Tourism,170.0,13473.0,980.0,2530.0,241.1
16432,PL020049,1.0,0.000,Max,11,5,Yearly,No,WorkPrivate,F,53.0,35.0,No,0,NaN,NaN,quslbttvcitxzeiy,7.0,Gasoline,Tourism,200.0,19225.0,1310.0,50.0,278.9
48635,PL001773,1.0,0.000,Max,10,5,Biannual,No,Retired,M,69.0,50.0,Yes,F,55.0,37.0,lqkdgbosdzrtitgx,5.0,Diesel,Tourism,200.0,22067.0,1291.0,100.0,176.6
74688,PL068982,2.0,0.000,Min,3,3,Biannual,No,WorkPrivate,M,57.0,36.0,No,0,NaN,NaN,cgkclpnidlmetsrb,22.0,Gasoline,Tourism,170.0,18750.0,900.0,340.0,113.8


In [ ]:
y_train.sample(n=4)

191015       0.00
83491        0.00
220582       0.00
68450     4275.02
Name: claim_amount, dtype: float64

# Training the model 🚀

You must first define your first function: `fit_model`. This function takes training data as arguments, and outputs a "model" object -- that you define as you wish. For instance, this could be an array of parameter values.

## Define your data preprocessing

<a name="data-preprocessing"></a>

You can add any class or function in this cell for preprocessing. Just make sure that you use the functions here in the `fit_model`, `predict_expected_claim` and `predict_premium` functions if necessary. *italicised text*

In [ ]:
%%aicrowd_include

# This magical command saves all code in this cell to a utils module.
# include your preprocessing functions and classes here.

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer


class NormalizeData:
    '''
    Class used to normalize a dataset according to a standard normal 
    distribution.

    Methods
    -------
    fit : Use the training dataset to calculate the mean and standard deviation
        used for the normalisation of new data.

    transform : Use the parameters calculated in the fit method to normalize 
        new data.
    '''
    def __init__(self):
        self.x_means = 0
        self.x_std = 0

    def fit(self, x_train):
        x_float = x_train.select_dtypes(include=['float', 'int']).drop(
            columns=['year', 'pol_no_claims_discount'])
        self.x_means = x_float.mean()
        self.x_std = x_float.std()
        return self

    def transform(self, x_raw):
        for idx in x_raw:
            if idx in self.x_means.index:
                x_raw[idx] = (x_raw[idx] - self.x_means[idx]) / self.x_std[idx]
        return x_raw


class Compress_vh_make_model:
    '''
    Class used to group the labels with low frequency from the feature 
    vh_make_model.

    Methods
    -------
    fit : Use the training dataset to calculate the mean and standard deviation
        used for the normalisation of new data.

    transform : Use the parameters calculated in the fit method to normalize 
        new data.
    '''
    def __init__(self, n_occurences=30):
        self.n_occ = n_occurences

    def fit(self, x_train):
        models_counts = x_train.vh_make_model.value_counts()
        self.models_to_group = models_counts[models_counts < self.n_occ].keys()
        return self
    
    def transform(self, x_raw):
        mask_model_to_group = x_raw.vh_make_model.isin(self.models_to_group)
        x_raw.loc[mask_model_to_group, 'vh_make_model'] = 'other_models'
        x_raw['rare_models'] = mask_model_to_group
        return x_raw


class Preprocess_X_data:
    """
    Class to preprocess the features of the dataset

    Methods
    -------
    add_new_features : Method to include new features

    impute_missing_values : Method to deal with missing values

    fit : Use the training data set to specify the parameters of the 
        prepocessing.

    transform : Use the parameters from the fit method to preprocess new data.

    """
    def __init__(self, n_occurences_vh_make_model=30, drop_id=False):
        self.normalizer = NormalizeData()
        self.compress_models = Compress_vh_make_model(
            n_occurences=n_occurences_vh_make_model
            )
        self.cols_to_binarize = ['pol_payd', 'drv_sex1', 'drv_drv2']
        self.cols_to_one_hot_encode = [
            'pol_coverage', 'pol_usage', 'drv_sex2', 
            'vh_make_model', 'vh_fuel', 'vh_type'
            ]
        self.drop_id = drop_id

    def add_new_features(self, x_raw):
        x = x_raw.copy()
        # Adding new features
        x.insert(
            loc=len(x.columns),
            column='pop_density',
            value = x.population / x.town_surface_area
            )
        x.insert(
            loc=len(x.columns),
            column='vh_speed_drv_age_ratio',
            value = x.vh_speed / x.drv_age1
            )
        x.insert(
            loc=len(x.columns),
            column='potential_force_impact',
            value = x.vh_speed * x.vh_weight
            )

        # Droping not necessay variables
        x = x.drop(columns='pol_pay_freq')
        if self.drop_id:
            x = x.drop(columns='id_policy')
        return x

    def impute_missing_values(self, x_raw):
        x = x_raw.copy()
        # Adding missing indicators
        x['vh_age_NA'] = x['vh_age'].isnull()
        x['vh_value_NA'] = x['vh_value'].isnull()

        # Impute missing values
        x = x.fillna(0)
        return x

    def fit(self, x_train):
        # Adding new features
        x_train = self.add_new_features(x_train)

        # Compressing the vh_make_model column
        self.compress_models.fit(x_train)
        x_train = self.compress_models.transform(x_train)

        # Normalization
        self.normalizer.fit(x_train)

        return self

    def transform(self, x_raw):
        # Adding new features
        x_prep = self.add_new_features(x_raw)

        # Compressing the vh_make_model column
        x_prep = self.compress_models.transform(x_prep)

        # Normalization
        colnames = x_prep.columns
        x_prep = self.normalizer.transform(x_prep)
        x_prep = pd.DataFrame(x_prep, columns=colnames)

        # Impute missing values
        x_prep = self.impute_missing_values(x_prep)

        # Binarize columns with only two categories
        lb = LabelBinarizer()
        for col in self.cols_to_binarize:
            x_prep[col] = lb.fit_transform(x_prep[col])

        # One-Hot-Encode the other categorical columns
        x_prep = pd.get_dummies(
            data=x_prep,
            prefix = self.cols_to_one_hot_encode,
            columns = self.cols_to_one_hot_encode,
            drop_first=True,
            dtype='int8'
            )

        return x_prep


In [ ]:
import importlib
import utils
importlib.reload(utils)
from utils import *  # do not change this

## Define the training logic

In [ ]:
def fit_model(X_raw, y_raw):
    """Model training function: given training data (X_raw, y_raw), train this pricing model.

    Parameters
    ----------
    X_raw : Pandas dataframe, with the columns described in the data dictionary.
        Each row is a different contract. This data has not been processed.
    y_raw : a Numpy array, with the value of the claims, in the same order as contracts in X_raw.
        A one dimensional array, with values either 0 (most entries) or >0.

    Returns
    -------
    self: this instance of the fitted model. This can be anything, as long as it is compatible
        with your prediction methods.

    """
    TRAINING_YEARS = [1,2,3]

    xgb_occ_params = {
        # Definition of the model to train
        "objective": ["binary:logistic"],
        "booster" : ['gbtree'],
        # Evaluation metric
        "eval_metric": ["auc"],
        # Parameters for gbtree booster
        'lambda':[1],
        "alpha": [1],
        'scale_pos_weight' : [5],
        'max_delta_step':[5],
        'tree_method':['gpu_hist'],
        # Additionnal parameters for the training function
        "early_stopping_rounds": [25],
        "num_boost_round": [4000]
    }

    xgb_sev_params = {
        # Definition of the model to train
        "objective": ["reg:tweedie"],
        "tweedie_variance_power" : [1.8],
        "booster" : ['gbtree'],
        # Evaluation metric
        "eval_metric": ["rmse"],
        # Parameters for gbtree booster
        'lambda':[1],
        "alpha": [1],
        'tree_method':['gpu_hist'],
        # Additionnal parameters for the training function
        "early_stopping_rounds": [25],
        "num_boost_round": [4000]
    }

    preprocessing = Preprocess_X_data(n_occurences_vh_make_model=50,
                                      drop_id=True)
    
    # Split the data in train and validation dataset according to the year
    x = X_raw.copy()

    train_mask = x.year.isin(TRAINING_YEARS).to_list()
    valid_mask = (~x.year.isin(TRAINING_YEARS)).to_list()

    x_train = x.loc[train_mask, :]
    y_train = y_raw[train_mask]
    x_valid = x.loc[valid_mask, :]
    y_valid = y_raw[valid_mask]

    # Preprocessing
    preprocessing.fit(x_train)
    x_train = preprocessing.transform(x_train)
    x_valid = preprocessing.transform(x_valid)

    # No more use of the column year
    x_train = x_train.drop(columns='year')
    x_valid = x_valid.drop(columns='year')


    def data_to_dmatrix(x_train, x_valid, y_train, y_valid):
        # Convert de features dataframes into DMatrix so it can be use to train an 
        # XGBoost
        dmatrix_train = xgb.DMatrix(x_train.values)
        dmatrix_train.set_label(y_train)
        dmatrix_valid = xgb.DMatrix(x_valid.values)
        dmatrix_valid.set_label(y_valid)
        return dmatrix_train, dmatrix_valid


    def params_dict_to_list(param_dict):
        # Transform xgb_params as a list of every combinations of parameters that 
        # needs to be tried during the gridsearch.
        keys, values = zip(*param_dict.items())
        param_list = [dict(zip(keys, v)) for v in itertools.product(*values)]
        print(f"Number of combinations of parameters to try: {len(param_list)}")
        return param_list


    def train_xgb(dmatrix_train, dmatrix_valid, param_list):
        # Train the XGBoost model
        results_list = list()
        for i, params_dict in enumerate(param_list):
            results_dict = {}
            model = xgb.train(
                params_dict,
                dtrain = dmatrix_train,
                num_boost_round = params_dict["num_boost_round"],
                early_stopping_rounds = params_dict["early_stopping_rounds"],
                evals = [(dmatrix_train, "train"), (dmatrix_valid, "eval")],
                evals_result = results_dict
            )
            results_list.append({
                "eval": float(list(results_dict["eval"].values())[0][-1]),
                "train": float(list(results_dict["train"].values())[0][-1]),
                "params": params_dict
            })
            print(f"Trained model #{i + 1} out of {len(param_list)}")
        print(results_list)
        return model
    
    # fit occurence model
    y_train_occ = y_train > 0
    y_valid_occ = y_valid > 0
    dmatrix_train, dmatrix_valid = data_to_dmatrix(
        x_train, x_valid, y_train_occ, y_valid_occ)
    params_list = params_dict_to_list(xgb_occ_params)
    trained_model_occ = train_xgb(dmatrix_train, dmatrix_valid, params_list)

    # fit severity model
    y_train_sev = y_train[y_train > 0]
    x_train_sev = x_train.loc[y_train > 0, :]
    y_valid_sev = y_valid[y_valid > 0]
    x_valid_sev = x_valid.loc[y_valid > 0, :]
    
    dmatrix_train, dmatrix_valid = data_to_dmatrix(
        x_train_sev, x_valid_sev, y_train_sev, y_valid_sev)
    params_list = params_dict_to_list(xgb_sev_params)
    trained_model_sev = train_xgb(dmatrix_train, dmatrix_valid, params_list)

    trained_model = [trained_model_occ, trained_model_sev]
    return trained_model


## Train your model

In [ ]:
trained_model = fit_model(X_train, y_train)

ValueError: ignored

**Important note**: your training code should be able to run in under 10 minutes (since this notebook is re-run entirely on the server side). 

If you run into an issue here we recommend using the *zip file submission* (see the [challenge page](https://www.aicrowd.com/challenges/insurance-pricing-game/#how-to%20submit)). In short, you can simply do this by copy-pasting your `fit_model`, `predict_expected_claim` and `predict_premium` functions to the `model.py` file.

Note that if you want to perform extensive cross-validation/hyper-parameter selection, it is better to do them offline, in a separate notebook.

## Saving your model

You can save your model to a file here, so you don't need to retrain it every time.

In [ ]:
def save_model(model_path):  # some models such xgboost models or keras models don't pickle very reliably. Please use the package provided saving functions instead. 
  with open(model_path, 'wb') as target_file:
      pickle.dump(trained_model, target_file)

In [ ]:
save_model(Config.MODEL_OUTPUT_PATH)

If you need to load it from file, you can use this code:

In [ ]:
def load_model(model_path): # some models such xgboost models or keras models don't pickle very reliably. Please use the package provided saving functions instead. 
  with open(model_path, 'rb') as target:
      return pickle.load(target)

In [ ]:
trained_model = load_model(Config.MODEL_OUTPUT_PATH)

# Predicting the claims 💵

The second function, `predict_expected_claim`, takes your trained model and a dataframe of contracts, and outputs a prediction for the (expected) claim incurred by each contract. This expected claim can be seen as the probability of an accident multiplied by the cost of that accident.

This is the function used to compute the _RMSE_ leaderboard, where the model best able to predict claims wins.

In [ ]:
def predict_expected_claim(model, X_raw):
    """Model prediction function: predicts the expected claim based on the pricing model.

    This functions estimates the expected claim made by a contract (typically, as the product
    of the probability of having a claim multiplied by the expected cost of a claim if it occurs),
    for each contract in the dataset X_raw.

    This is the function used in the RMSE leaderboard, and hence the output should be as close
    as possible to the expected cost of a contract.

    Parameters
    ----------
    model: a Python object that describes your model. This can be anything, as long
        as it is consistent with what `fit` outpurs.
    X_raw : Pandas dataframe, with the columns described in the data dictionary.
        Each row is a different contract. This data has not been processed.

    Returns
    -------
    avg_claims: a one-dimensional Numpy array of the same length as X_raw, with one
        expected claim per contract (in same order). These expected claims must be POSITIVE (>0).
    """
    
    # Preprocessing
    x = X_raw.copy()
    preprocessing = Preprocess_X_data(
        n_occurences_vh_make_model=50,
        drop_id=False
        )
    preprocessing.fit(X_train[X_train.year.isin([1,2,3])])
    x_clean = preprocessing.transform(x)
    x_clean = x_clean.drop(columns='year')
    policy_id = x_clean.pop('id_policy')
    x_clean = xgb.DMatrix(x_clean.values)

    # predictions
    frequency_model, severity_model = model

    expected_frequency = frequency_model.predict(x_clean)
    expected_severity = severity_model.predict(x_clean)

    expected_claim = expected_frequency * expected_severity

    return expected_claim

To test your function, run it on your training data:

In [ ]:
expected_claims = predict_expected_claim(trained_model, X_train)

In [ ]:
((expected_claims - y_train)**2).mean() ** 0.5

751.8928092363773

# Pricing contracts 💰💰

The third and final function, `predict_premium`, takes your trained model and a dataframe of contracts, and outputs a _price_ for each of these contracts. **You are free to set this prices however you want!** These prices will then be used in competition with other models: contracts will choose the model offering the lowest price, and this model will have to pay the cost if an accident occurs.

This is the function used to compute the _profit_ leaderboard: your model will participate in many markets of size 10, populated by other participants' model, and we compute the average profit of your model over all the markets it participated in.

In [ ]:
def predict_premium(model, X_raw):
    """Model prediction function: predicts premiums based on the pricing model.

    This function outputs the prices that will be offered to the contracts in X_raw.
    premium will typically depend on the average claim predicted in 
    predict_average_claim, and will add some pricing strategy on top.

    This is the function used in the average profit leaderboard. Prices output here will
    be used in competition with other models, so feel free to use a pricing strategy.

    Parameters
    ----------
    model: a Python object that describes your model. This can be anything, as long
        as it is consistent with what `fit` outpurs.
    X_raw : Pandas dataframe, with the columns described in the data dictionary.
        Each row is a different contract. This data has not been processed.

    Returns
    -------
    prices: a one-dimensional Numpy array of the same length as X_raw, with one
        price per contract (in same order). These prices must be POSITIVE (>0).
    """

    # TODO: return a price for everyone.
    # Don't forget any preprocessing of the raw data here

    return predict_expected_claim(model, X_raw) * 2  # Default: bosst prices by a factor of 2

To test your function, run it on your training data.

In [ ]:
prices = predict_premium(trained_model, X_train)

#### Profit on training data

In order for your model to be considered in the profit competition, it needs to make nonnegative profit over its training set. You can check that your model satisfies this condition below:

In [ ]:
print('Income:', prices.sum())
print('Losses:', y_train.sum())

if prices.sum() < y_train.sum():
    print('Your model loses money on the training data! It does not satisfy market rule 1: Non-negative training profit.')
    print('This model will be disqualified from the weekly profit leaderboard, but can be submitted for educational purposes to the RMSE leaderboard.')
else:
    print('Your model passes the non-negative training profit test!')

# Ready? Submit to AIcrowd 🚀

If you are satisfied with your code, run the code below to send your code to the AICrowd servers for evaluation! This requires the variable `trained_model` to be defined by your previous code.

**Make sure you have included all packages needed to run your code in the [_"Packages"_](#packages) section.**

**NOTE**: If you submit the baseline RMSE model without any change whatsoever, your model will not be entered into the market. 

In [ ]:
%aicrowd_submit